# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de dictionnaires : chaque phrase a une clé "mots" qui est associée à une liste des mots, et une clé "POS" associée à une liste de POS. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [1]:
def load_corpus(file):
    with open(file, "r") as f:
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_dico = {"mots" : [], "POS" : []} # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                phrase_dico["mots"].append(features[1])
                phrase_dico["POS"].append(features[3])
        corpus.append(phrase_dico)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [2]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[100], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

{'mots': ["L'", 'œuvre', 'est', 'située', 'dans', 'la', 'galerie', 'des', 'de', 'les', 'batailles', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.'], 'POS': ['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT']}

{'mots': ['Ce', 'résultat', 'provient', 'à', 'hauteur', 'de', '18,5', 'ME', 'de', 'la', 'progression', 'de', 'la', 'rentabilité', 'de', "l'", 'habitat', 'de', 'loisirs', 'qui', 'dégage', 'une', 'marge', 'opérationnelle', 'de', '9,5', '%', 'contre', '8,5', '%', "l'", 'année', 'passée', '.'], 'POS': ['DET', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'NUM', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'NUM', 'SYM', 'ADP', 'NUM', 'SYM', 'DET', 'NOUN', 'ADJ', 'PUNCT']}

{'mots': ['Cette', 'espèce', 'est', 'endémique', 'du', 'de', 'le', 'département', 'de', 'Nariño'

## Extraction des caractéristiques
Pour clarifier le pipeline, nous allons ajouter à notre corpus des nouvelles entrées dans les dictionnaires de chaque phrase afin de stocker les caractéristiques suivantes :
- mot
- mot précédent
- mot suivant
- commence par une lettre majuscule
- est numérique
- contient des caractères non alphanumériques
- longueur du mot

In [7]:
def feature_extraction(corpus):
    
    # clés des dictionnaires
    mots = "mots"
    pos = "POS"
    mot_prec = "préc"
    mot_suiv = "suiv"
    maj = "maj"
    num = "nb"
    nonAlphanum = "non alphanum"
    long = "longueur"

    corpus_features = corpus.copy() # copie de la liste pour ne pas modifier la liste d'origine

    for phrase in corpus_features: # ajout des features additionnelles
        phrase.update({mot_prec : [], mot_suiv : [], maj : [], num : [], nonAlphanum : [], long : []})
        for prev, word, suiv in zip(["START"] + phrase["mots"][:-1], phrase["mots"], phrase["mots"][1:] + ["END"]):
            # création de triplets (mot précédent, mot, mot suivant)
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier
            phrase[mot_prec].append(prev)
            phrase[mot_suiv].append(suiv)
            phrase[maj].append(word.istitle())
            phrase[num].append(word.isnumeric())
            phrase[nonAlphanum].append(not word.isalnum)
            phrase[long].append(len(word))

    return corpus_features

gsd_train_features = feature_extraction(gsd_train)
print(gsd_train_features[4], sep="\n")

{'mots': ['Ismene', 'entre', 'et', 'annonce', 'que', "c'", 'est', 'Farnace', 'qui', 'a', 'mis', 'le', 'feu', 'à', 'la', 'flotte', 'romaine', '.'], 'POS': ['PROPN', 'VERB', 'CCONJ', 'VERB', 'SCONJ', 'PRON', 'AUX', 'PROPN', 'PRON', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT'], 'préc': ['START', 'Ismene', 'entre', 'et', 'annonce', 'que', "c'", 'est', 'Farnace', 'qui', 'a', 'mis', 'le', 'feu', 'à', 'la', 'flotte', 'romaine'], 'suiv': ['entre', 'et', 'annonce', 'que', "c'", 'est', 'Farnace', 'qui', 'a', 'mis', 'le', 'feu', 'à', 'la', 'flotte', 'romaine', '.', 'END'], 'maj': [True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False], 'nb': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False], 'non alphanum': [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

## Implémentation de l'algorithme de classification
On a choisi d'implémenter la classification avec un perceptron moyenné.

## Evaluation des performances sur le corpus de validation (Dev)
Il faudra bien tester les hyper paramètres (epoch...)

## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine